In [1]:
import sys
import os 

paths = ['../src/', '../src/api/v6']
for path in paths:
    path = os.path.normcase(path)
    if not any(os.path.normcase(sp) == path for sp in sys.path):
        sys.path.append(path)

In [3]:
text = """
    [ Module
		[ source = Location.resolve_from_text('Walmart') ]
		[ destination = Location.resolve_from_text('Walgreens') ]
		[ directions = Navigation.find_directions(source=source, destination=destination, date_time=date_time) ]
		[ Responder.respond(response=directions) ]
		[ recipient1 = Contact.resolve_from_text('Pepsi') ]
		[ recipient2 = Contact.resolve_from_text('Walgreens') ]
		[ For
			[ test
				[ iter
					[ 
						[ recipient1, recipient2 ]
					]
				]
				[ Name
					[ recipient ]
				]
			]
			[ body
				[ exact_content = Content.resolve_from_text('check it out') ]
				[ Message.send_message(recipient=recipient, exact_content=exact_content) ]
			]
		]
	]
 """

In [4]:
import ast
from representations.tree.tree import Tree
from representations.builders.ast.tearers.tearer_factory import TearerFactory

In [5]:
tree = Tree.unparse(text)
print(tree)

	[ Module
		[ source = Location.resolve_from_text('Walmart') ]
		[ destination = Location.resolve_from_text('Walgreens') ]
		[ directions = Navigation.find_directions(source=source, destination=destination, date_time=date_time) ]
		[ Responder.respond(response=directions) ]
		[ recipient1 = Contact.resolve_from_text('Pepsi') ]
		[ recipient2 = Contact.resolve_from_text('Walgreens') ]
		[ For
			[ test
				[ iter
					[ [ recipient1, recipient2 ] ]
				]
				[ Name
					[ recipient ]
				]
			]
			[ body
				[ exact_content = Content.resolve_from_text('check it out') ]
				[ Message.send_message(recipient=recipient, exact_content=exact_content) ]
			]
		]
	]



In [6]:
tearer = TearerFactory().get_tearer(tree.root_node)
asdl2 = tearer.tear(tree.root_node)
x = ast.unparse(asdl2)
print(x)

source = Location.resolve_from_text('Walmart')
destination = Location.resolve_from_text('Walgreens')
directions = Navigation.find_directions(source=source, destination=destination, date_time=date_time)
Responder.respond(response=directions)
recipient1 = Contact.resolve_from_text('Pepsi')
recipient2 = Contact.resolve_from_text('Walgreens')
for recipient in [recipient1, recipient2]:
    exact_content = Content.resolve_from_text('check it out')
    Message.send_message(recipient=recipient, exact_content=exact_content)


In [7]:
text = """
[ Module [ message_content_type = MessageContentType(text='a text message') ] [ sender = Contact(text='my boss Tony') ] [ messages = Messages.find_messages(sender=sender, message_content_type=message_content_type) ] [ test_messages = bool(messages) ] [ If [ test [ test_messages ] ] [ body [ senders = Contact.resolve_many_from_text(text='work') ] [ message_content_type = MessageContentType.resolve_from_text(text='any emails') ] [ messages = [] ] [ For [ test [ iter [ senders ] ] [ Name [ sender ] ] ] [ body [ messages += Messages.find_messages(sender=sender, message_content_type=message_content_type) ] ] ] [ Responder.respond(response=messages) ] ] ] ]
"""
tree = Tree.unparse(text)
tearer = TearerFactory().get_tearer(tree.root_node)
asdl2 = tearer.tear(tree.root_node)
x = ast.unparse(asdl2)
print(x)

message_content_type = MessageContentType(text='a text message')
sender = Contact(text='my boss Tony')
messages = Messages.find_messages(sender=sender, message_content_type=message_content_type)
test_messages = bool(messages)
if test_messages:
    senders = Contact.resolve_many_from_text(text='work')
    message_content_type = MessageContentType.resolve_from_text(text='any emails')
    messages = []
    for sender in senders:
        messages += Messages.find_messages(sender=sender, message_content_type=message_content_type)
    Responder.respond(response=messages)


In [20]:
import pandas as pd

df = pd.read_csv('../build/eval_complex_utterance_to_code_with_intermediate_78_20230511.csv.gz')

In [21]:
df.head()

,test_id,sample_id,sample_minor_id,text,code,test,imports,lang_rep,code_rep
0,1_a,1,a,Check the availability of Pepsi at Walmart and...,"product_name = ProductName.resolve_from_text(""...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Action [ hd [ Check ] ]...,[ Module [ product_name = ProductName.resolve_...
1,1_b,1,b,"If it's raining tomorrow morning, set my alarm...","date_time = DateTime.resolve_from_text(""tomorr...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Condition [ If [ Test [...,[ Module [ date_time = DateTime.resolve_from_t...
2,2,2,NaN,"If it's raining tomorrow morning, set my alarm...","date_time = DateTime.resolve_from_text(""tomorr...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Condition [ If [ Test [...,[ Module [ date_time = DateTime.resolve_from_t...
3,3_a,3,a,Play the new Taylor Swift album and pull up my...,"album = Album.resolve_from_text(""the new Taylo...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Action [ hd [ Play ] ] ...,[ Module [ album = Album.resolve_from_text('th...
4,3_b,3,b,Send a message to dad if it rains tomorrow.,"date_time = DateTime.resolve_from_text(""tomorr...",# test data\ndata_model = DataModel(reset=True...,from entities.generic import *\nfrom entities....,[ root [ S [ Command [ Condition [ If [ Body [...,[ Module [ date_time = DateTime.resolve_from_t...


In [9]:
df['text'][50:60]

50    For Shakey Graves' upcoming summer tour, what ...
51    If I get a text message from my boss Tony, the...
52    If I get a text message from my boss Tony, the...
53    Check the weather for next tuesday and create ...
54    Message my brother I will not be able to make ...
55    Set a timer for 3:00 PM then enable home secur...
56    If it starts raining between 3pm-5pm, turn the...
57    Show me a map of downtown Phoenix and give me ...
58    Play my lofi Spotify playlist and buy tickets ...
59    Check if it's supposed to rain tonight and if ...
Name: text, dtype: object

In [10]:
df.iloc[0]['text']

'Check the availability of Pepsi at Walmart and also check it at Walgreens.'

In [11]:
df.iloc[0]['code_rep']

"[ Module [ product_name = ProductName.resolve_from_text('Pepsi') ] [ location = Location.resolve_from_text('Walmart') ] [ products = Shopping.find_products(product_name=product_name, location=location) ] [ Responder.respond(response=products) ] [ location = Location.resolve_from_text('Walgreens') ] [ products = Shopping.find_products(product_name=product_name, location=location) ] [ Responder.respond(response=products) ] ]"

In [12]:
tree = Tree.unparse(df.iloc[0]['code_rep'])

In [13]:
tearer = TearerFactory().get_tearer(tree.root_node)
asdl2 = tearer.tear(tree.root_node)
x = ast.unparse(asdl2)
print(x)

product_name = ProductName.resolve_from_text('Pepsi')
location = Location.resolve_from_text('Walmart')
products = Shopping.find_products(product_name=product_name, location=location)
Responder.respond(response=products)
location = Location.resolve_from_text('Walgreens')
products = Shopping.find_products(product_name=product_name, location=location)
Responder.respond(response=products)


In [14]:
import utils.api_utils

In [15]:
for row in df.iterrows():
    print(row[1]['text'])

Check the availability of Pepsi at Walmart and also check it at Walgreens.
If it's raining tomorrow morning, set my alarm for 7:30, if it's not, set my alarm for 8.
Play the new Taylor Swift album and pull up my shopping list for today.
Send a message to dad if it rains tomorrow.
Give me directions to Navy pier in Chicago and tell me what the current traffic is looking like.
Check the next time Blink 182 will be in Chicago and tell me the ticket prices.
Remind me tomorrow to email Jim about lunch and schedule a reservation for noon at the cafe.
Can you place an order for two turkeys to arrive the 22nd, and remind me about it on the 21st?
How long will it take to get to AMC theater at 8pm tonight and tell me what tonight's weather outlook is.
Check the weather for the 4th of July and send a text to Grandpa to invite him over and tell him the weather.
Set a timer for one hour and text Stacy that dinner will be ready in one hour.
If the weather is cold tomorrow please remind me to grab my

In [16]:
text = df.iloc[-1]['code_rep']
text

"[ Module [ event_names = EventName.resolve_many_from_text('free events') ] [ date_times = DateTime.resolve_many_from_text('this weekend') ] [ events = [] ] [ For [ test [ iter [ event_names ] ] [ Name [ event_name ] ] ] [ body [ For [ test [ iter [ date_times ] ] [ Name [ date_time ] ] ] [ body [ events += Calendar.find_events(event_name=event_name, date_time=date_time) ] ] ] ] ] [ Responder.respond(response=events) ] [ weather_forecasts = Weather.find_weather_forecasts() ] [ Responder.respond(response=weather_forecasts) ] ]"

In [17]:
print(df.iloc[-1]['code'])

event_names = EventName.resolve_many_from_text("free events")
date_times = DateTime.resolve_many_from_text("this weekend")
events = []
for event_name in event_names:
    for date_time in date_times:
        events += Calendar.find_events(event_name=event_name, date_time=date_time)
Responder.respond(response=events)

weather_forecasts = Weather.find_weather_forecasts()
Responder.respond(response=weather_forecasts)



In [18]:
tree = Tree.unparse(text)
print(tree)

	[ Module
		[ event_names = EventName.resolve_many_from_text('free events') ]
		[ date_times = DateTime.resolve_many_from_text('this weekend') ]
		[ events = [] ]
		[ For
			[ test
				[ iter
					[ event_names ]
				]
				[ Name
					[ event_name ]
				]
			]
			[ body
				[ For
					[ test
						[ iter
							[ date_times ]
						]
						[ Name
							[ date_time ]
						]
					]
					[ body
						[ events += Calendar.find_events(event_name=event_name, date_time=date_time) ]
					]
				]
			]
		]
		[ Responder.respond(response=events) ]
		[ weather_forecasts = Weather.find_weather_forecasts() ]
		[ Responder.respond(response=weather_forecasts) ]
	]



In [19]:
tearer = TearerFactory().get_tearer(tree.root_node)
asdl2 = tearer.tear(tree.root_node)
x = ast.unparse(asdl2)
print(x)

event_names = EventName.resolve_many_from_text('free events')
date_times = DateTime.resolve_many_from_text('this weekend')
events = []
for event_name in event_names:
    for date_time in date_times:
        events += Calendar.find_events(event_name=event_name, date_time=date_time)
Responder.respond(response=events)
weather_forecasts = Weather.find_weather_forecasts()
Responder.respond(response=weather_forecasts)


In [60]:
from main_representations import (
    generate_text_representation,
    generate_code_representation,
)

code = df.iloc[-1]['code']

rep_code = generate_code_representation(
    code, rules_enabled=False
)



In [65]:
print(rep_code)

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x106248be0>>


TypeError: __repr__ returned non-string (type Node)

In [27]:
with open('/Users/asaf/Workspace/biu/complex-utterance-to-code/tests/api/eval/test_block_01.py', "r") as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        print(i, line)
        
        if 'def ' in lines[i+1]:
            break

0 from entities.generic import *

1 from entities.calendar import *

2 from entities.home import *

3 from entities.map import *

4 from entities.message import *

5 from entities.music import *

6 from entities.navigation import *

7 from entities.reminder import *

8 from entities.shopping import *

9 from entities.weather import *

10 from actions.calendar import *

11 from actions.clock import *

12 from actions.calendar import *

13 from actions.home import *

14 from actions.map import *

15 from actions.messages import *

16 from actions.music import *

17 from actions.navigation import *

18 from actions.reminders import *

19 from actions.responder import *

20 from actions.shopping import *

21 from actions.weather import *

22 from providers.data_model import DataModel

23 from datetime import datetime, timedelta

24 import utils.api_utils as utils

25 from utils.test_utils import *

26 

27 

